In [5]:
# --------------------------------------------------------------
#  1. MOUNT & IMPORT
# --------------------------------------------------------------
from google.colab import drive
import os
from pathlib import Path
import shutil

# Clean the mount directory if it exists
mount_point = '/content/drive'
if os.path.exists(mount_point):
    print(f"Cleaning {mount_point}...")
    # Remove all files and directories in /content/drive
    for item in Path(mount_point).iterdir():
        if item.is_file():
            item.unlink()
        elif item.is_dir():
            shutil.rmtree(item)
    print("Mount point cleaned")

# Now mount fresh
drive.mount('/content/drive')

# --------------------------------------------------------------
#  2. CREATE SYMBOLIC LINKS
# --------------------------------------------------------------
MERGED_ROOT = "/content/merged_dataset"

# Clear and recreate merged dataset
if os.path.exists(MERGED_ROOT):
    shutil.rmtree(MERGED_ROOT)
os.makedirs(MERGED_ROOT, exist_ok=True)

def create_symlink_folder(src_folder, target_folder, prefix=""):
    if not Path(src_folder).exists():
        print(f"⚠️  Skipping {src_folder} - folder not found")
        return 0

    os.makedirs(target_folder, exist_ok=True)
    files_linked = 0

    for item in Path(src_folder).iterdir():
        if item.is_file():
            # Direct file in folder (Nothing_2a structure)
            if prefix:
                new_name = f"{prefix}_{item.name}"
            else:
                new_name = item.name
            link_path = Path(target_folder) / new_name
            if not link_path.exists():
                os.symlink(item, link_path)
                files_linked += 1
        elif item.is_dir():
            # Subfolder with files (NTU structure)
            for file_item in item.iterdir():
                if file_item.is_file():
                    # Create unique name using folder name
                    if prefix:
                        unique_name = f"{prefix}_{item.name}_{file_item.name}"
                    else:
                        unique_name = f"{item.name}_{file_item.name}"
                    link_path = Path(target_folder) / unique_name
                    if not link_path.exists():
                        os.symlink(file_item, link_path)
                        files_linked += 1

    return files_linked

print(f"Creating virtual merge at: {MERGED_ROOT}")

# Define source folders
ntu_orig = "/content/drive/MyDrive/NTU-Roselab-Dataset/originals"
ntu_recp = "/content/drive/MyDrive/NTU-Roselab-Dataset/recaptures"
mob_orig = "/content/drive/MyDrive/Nothing_2a/originals"
mob_recp = "/content/drive/MyDrive/Nothing_2a/recaptures"

# Create merged structure with prefixes to avoid conflicts
print("Linking files...")
ntu_orig_count = create_symlink_folder(ntu_orig, f"{MERGED_ROOT}/originals", "ntu")
mob_orig_count = create_symlink_folder(mob_orig, f"{MERGED_ROOT}/originals", "nothing")
ntu_recp_count = create_symlink_folder(ntu_recp, f"{MERGED_ROOT}/recaptures", "ntu")
mob_recp_count = create_symlink_folder(mob_recp, f"{MERGED_ROOT}/recaptures", "nothing")

print(f"\nLinking results:")
print(f"NTU Originals: {ntu_orig_count} files")
print(f"Nothing_2a Originals: {mob_orig_count} files")
print(f"NTU Recaptures: {ntu_recp_count} files")
print(f"Nothing_2a Recaptures: {mob_recp_count} files")

print(f"\nFinal merged dataset:")
print(f"Total Originals: {len(list(Path(f'{MERGED_ROOT}/originals').glob('*')))} files")
print(f"Total Recaptures: {len(list(Path(f'{MERGED_ROOT}/recaptures').glob('*')))} files")

# Show sample files from each source
print(f"\nSample NTU originals:")
ntu_originals = [f for f in Path(f'{MERGED_ROOT}/originals').glob('ntu_*')]
for f in ntu_originals[:5]:
    print(f"  {f.name}")

print(f"\nSample Nothing_2a originals:")
nothing_originals = [f for f in Path(f'{MERGED_ROOT}/originals').glob('nothing_*')]
for f in nothing_originals[:5]:
    print(f"  {f.name}")

print(f"\nSample NTU recaptures:")
ntu_recaptures = [f for f in Path(f'{MERGED_ROOT}/recaptures').glob('ntu_*')]
for f in ntu_recaptures[:5]:
    print(f"  {f.name}")

print(f"\nSample Nothing_2a recaptures:")
nothing_recaptures = [f for f in Path(f'{MERGED_ROOT}/recaptures').glob('nothing_*')]
for f in nothing_recaptures[:5]:
    print(f"  {f.name}")

print(f"\n✅ Dataset merging completed!")
print(f"📍 Merged dataset location: {MERGED_ROOT}")
print(f"📁 Originals folder: {MERGED_ROOT}/originals")
print(f"📁 Recaptures folder: {MERGED_ROOT}/recaptures")

Cleaning /content/drive...
Mount point cleaned
Mounted at /content/drive
Creating virtual merge at: /content/merged_dataset
Linking files...

Linking results:
NTU Originals: 1202 files
Nothing_2a Originals: 106 files
NTU Recaptures: 1199 files
Nothing_2a Recaptures: 103 files

Final merged dataset:
Total Originals: 1308 files
Total Recaptures: 1302 files

Sample NTU originals:
  ntu_MjuAcer_013.JPG
  ntu_OlymNec_042.JPG
  ntu_OlympusE500_070.JPG
  ntu_MjuNec_071.JPG
  ntu_OlymNec_041.JPG

Sample Nothing_2a originals:
  nothing_IMG_20251002_160015732.jpg
  nothing_IMG_20251007_190319031.jpg
  nothing_IMG_20250726_190213518.jpg
  nothing_IMG_20251002_160021521.jpg
  nothing_IMG_20251007_174427204.jpg

Sample NTU recaptures:
  ntu_zLumixD1Outdoor_071.jpg
  ntu_zCasioIndoor_076.JPG
  ntu_zCanonIxusIndoor_021.JPG
  ntu_zLumixD1Indoor_063.jpg
  ntu_zLumixD1Outdoor_063.jpg

Sample Nothing_2a recaptures:
  nothing_IMG_20251013_023758169.jpg
  nothing_IMG_20251013_023741061.jpg
  nothing_IMG_20

Downloading Model


In [7]:
# --------------------------------------------------------------
#  DOWNLOAD EfficientNetB0 weights ONCE & SAVE TO DRIVE
# --------------------------------------------------------------
import urllib.request
import os
from pathlib import Path

# URL and local path
URL = "https://storage.googleapis.com/keras-applications/efficientnetb0_notop.h5"
SAVE_PATH = "/content/drive/MyDrive/efficientnetb0_notop.h5"

# Download with progress bar
print("Downloading EfficientNetB0 weights (~16MB)...")
def download_with_progress(url, filepath):
    if os.path.exists(filepath):
        print(f"Already exists: {filepath}")
        return filepath
    with urllib.request.urlopen(url) as response, open(filepath, 'wb') as out_file:
        length = int(response.getheader('content-length', 0))
        downloaded = 0
        chunk_size = 1024 * 1024
        while True:
            data = response.read(chunk_size)
            if not data:
                break
            out_file.write(data)
            downloaded += len(data)
            if length > 0:
                print(f"\rProgress: {downloaded / length:.1%}", end="")
    print(f"\nSaved to: {filepath}")
    return filepath

try:
    download_with_progress(URL, SAVE_PATH)
except Exception as e:
    print("Failed. Trying alternative mirror...")
    # Alternative mirror (GitHub)
    URL_ALT = "https://github.com/keras-team/keras-applications/releases/download/efficientnet/efficientnetb0_notop.h5"
    download_with_progress(URL_ALT, SAVE_PATH)

print("Weights ready for local loading!")

Progress: 100.0%
Saved to: /content/drive/MyDrive/efficientnetb0_notop.h5
Weights ready for local loading!


## Replace Model Creation with Local Weight Loading

In [8]:
# --------------------------------------------------------------
#  MODEL – LOAD WEIGHTS FROM DRIVE (NO INTERNET NEEDED AFTER)
# --------------------------------------------------------------
WEIGHTS_PATH = "/content/drive/MyDrive/efficientnetb0_notop.h5"

def create_model():
    rgb_in = Input((IMG_SIZE, IMG_SIZE, 3), name='rgb')
    lap_in = Input((IMG_SIZE, IMG_SIZE, 3), name='lap')

    # Use weights=None → we'll load manually
    b1 = applications.EfficientNetB0(include_top=False, weights=None, name='eff_rgb')(rgb_in)
    b1 = GlobalAveragePooling2D()(b1)
    b2 = applications.EfficientNetB0(include_top=False, weights=None, name='eff_lap')(lap_in)
    b2 = GlobalAveragePooling2D()(b2)

    x = Concatenate()([b1, b2])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    out = Dense(1, activation='sigmoid')(x)

    m = Model([rgb_in, lap_in], out)
    m.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

    # Load weights from local file
    print("Loading EfficientNet weights from Drive...")
    m.get_layer('eff_rgb').load_weights(WEIGHTS_PATH, by_name=True, skip_mismatch=True)
    m.get_layer('eff_lap').load_weights(WEIGHTS_PATH, by_name=True, skip_mismatch=True)
    print("Weights loaded!")

    return m

model = create_model()

Loading EfficientNet weights from Drive...
Weights loaded!


## Merged Model

In [3]:
# --------------------------------------------------------------
# 0. MOUNT DRIVE
# --------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# --------------------------------------------------------------
# 1. IMPORTS
# --------------------------------------------------------------
import os, json, urllib.request, numpy as np, tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications, callbacks, layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import image_dataset_from_directory
from PIL import Image

# --------------------------------------------------------------
# 2. GLOBAL SETTINGS
# --------------------------------------------------------------
MERGED_ROOT      = "/content/merged_dataset"
WEIGHTS_PATH     = "/content/drive/MyDrive/efficientnetb0_notop.h5"
IMG_SIZE         = 224
BATCH_SIZE       = 16
EPOCHS           = 15
OUTPUT_DIR       = "/content/drive/MyDrive/Merged_Model/results"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

CHECKPOINT_DIR   = OUTPUT_DIR                     # full-model files live here
CHECKPOINT_PTR   = f"{OUTPUT_DIR}/latest_checkpoint.json"
HISTORY_FILE     = f"{OUTPUT_DIR}/history.json"

# --------------------------------------------------------------
# 3. (ONE-TIME) DOWNLOAD EFFICIENTNET WEIGHTS
# --------------------------------------------------------------
def download_weights():
    if os.path.exists(WEIGHTS_PATH):
        print(f"Weights ready: {WEIGHTS_PATH}")
        return
    print("Downloading EfficientNetB0 weights...")
    url = "https://github.com/keras-team/keras-applications/releases/download/efficientnet/efficientnetb0_notop.h5"
    urllib.request.urlretrieve(url, WEIGHTS_PATH)
    print("Saved to Drive")

download_weights()

# --------------------------------------------------------------
# 4. AUTO-REBUILD merged_dataset (runs every session)
# --------------------------------------------------------------
def build_merged_dataset():
    if os.path.isdir(MERGED_ROOT) and any(Path(MERGED_ROOT).iterdir()):
        print(f"Using existing merged dataset: {MERGED_ROOT}")
        return
    print(f"Building merged dataset at: {MERGED_ROOT}")

    ntu_orig = "/content/drive/MyDrive/NTU-Roselab-Dataset/originals"
    ntu_recp = "/content/drive/MyDrive/NTU-Roselab-Dataset/recaptures"
    mob_orig = "/content/drive/MyDrive/Nothing_2a/originals"
    mob_recp = "/content/drive/MyDrive/Nothing_2a/recaptures"

    def link_folder(src, dst, prefix):
        if not Path(src).exists():
            print(f"Warning: Skipping {src} (not found)")
            return 0
        os.makedirs(dst, exist_ok=True)
        cnt = 0
        for item in Path(src).iterdir():
            if item.is_file():
                name = f"{prefix}_{item.name}"
                link = Path(dst) / name
                if not link.exists():
                    os.symlink(item, link)
                    cnt += 1
        return cnt

    os.makedirs(MERGED_ROOT, exist_ok=True)
    orig = link_folder(ntu_orig, f"{MERGED_ROOT}/originals", "ntu")
    orig += link_folder(mob_orig, f"{MERGED_ROOT}/originals", "nothing")
    recp = link_folder(ntu_recp, f"{MERGED_ROOT}/recaptures", "ntu")
    recp += link_folder(mob_recp, f"{MERGED_ROOT}/recaptures", "nothing")
    print(f"Merged: {orig} originals, {recp} recaptures")

build_merged_dataset()

# --------------------------------------------------------------
# 5. PRE-PROCESSING
# --------------------------------------------------------------
@tf.function
def rgb_preprocess(img):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    return tf.keras.applications.efficientnet.preprocess_input(img)

@tf.function
def laplacian_preprocess(img):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.image.rgb_to_grayscale(img)
    kernel = tf.constant([[0,-1,0],[-1,4,-1],[0,-1,0]], dtype=tf.float32)
    kernel = tf.reshape(kernel, [3,3,1,1])
    filtered = tf.nn.conv2d(img, kernel, strides=1, padding='SAME')
    filtered = tf.abs(filtered)
    mn, mx = tf.reduce_min(filtered), tf.reduce_max(filtered)
    filtered = (filtered - mn) / (mx - mn + 1e-6)
    return tf.tile(filtered, [1,1,1,3])

@tf.function
def preprocess_dual(img, label):
    return (rgb_preprocess(img), laplacian_preprocess(img)), label

@tf.function
def augment(inputs, label):
    rgb, lap = inputs
    rgb = tf.image.random_flip_left_right(rgb)
    lap = tf.image.random_flip_left_right(lap)
    k = tf.random.uniform([], 0, 4, tf.int32)
    rgb = tf.image.rot90(rgb, k)
    lap = tf.image.rot90(lap, k)
    return (rgb, lap), label

# --------------------------------------------------------------
# 6. BUILD DATASETS
# --------------------------------------------------------------
train_ds = image_dataset_from_directory(
    MERGED_ROOT,
    label_mode='binary',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='training',
    seed=42,
    shuffle=True
).map(preprocess_dual).map(augment).prefetch(tf.data.AUTOTUNE)

val_ds = image_dataset_from_directory(
    MERGED_ROOT,
    label_mode='binary',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='validation',
    seed=42
).map(preprocess_dual).prefetch(tf.data.AUTOTUNE)

print(f"Train batches: {len(train_ds)} | Val batches: {len(val_ds)}")

# --------------------------------------------------------------
# 7. MODEL + RESUME LOGIC
# --------------------------------------------------------------
def create_model():
    rgb_in = Input((IMG_SIZE, IMG_SIZE, 3), name='rgb')
    lap_in = Input((IMG_SIZE, IMG_SIZE, 3), name='lap')

    b1 = applications.EfficientNetB0(include_top=False, weights=None, name='eff_rgb')(rgb_in)
    b1 = GlobalAveragePooling2D()(b1)
    b2 = applications.EfficientNetB0(include_top=False, weights=None, name='eff_lap')(lap_in)
    b2 = GlobalAveragePooling2D()(b2)

    x = Concatenate()([b1, b2])
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    out = Dense(1, activation='sigmoid')(x)

    m = Model([rgb_in, lap_in], out)
    m.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Precision', 'Recall'])

    print("Loading EfficientNet backbone from Drive...")
    m.get_layer('eff_rgb').load_weights(WEIGHTS_PATH, by_name=True, skip_mismatch=True)
    m.get_layer('eff_lap').load_weights(WEIGHTS_PATH, by_name=True, skip_mismatch=True)
    print("Backbone loaded")
    return m

def load_latest_checkpoint():
    """Return (model, start_epoch, history_dict)"""
    if not os.path.exists(CHECKPOINT_PTR):
        print("No checkpoint → starting fresh")
        return create_model(), 0, {}

    with open(CHECKPOINT_PTR) as f:
        info = json.load(f)
    cp_path = info['path']
    epoch   = info['epoch']          # 0-based
    print(f"Resuming from {cp_path} (epoch {epoch+1})")
    model = tf.keras.models.load_model(cp_path)

    hist = {}
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE) as f:
            hist = json.load(f)
    return model, epoch, hist

model, start_epoch, history_dict = load_latest_checkpoint()
print(f"Training from epoch {start_epoch+1}/{EPOCHS}")

# --------------------------------------------------------------
# 8. CALLBACK – SAVE FULL MODEL + POINTER + HISTORY
# --------------------------------------------------------------
class FullCheckpoint(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        epoch += 1                      # make 1-based for filename
        cp_path = f"{CHECKPOINT_DIR}/epoch_{epoch:02d}.keras"
        self.model.save(cp_path, overwrite=True)

        # update pointer file
        with open(CHECKPOINT_PTR, 'w') as f:
            json.dump({'path': cp_path, 'epoch': epoch-1}, f)   # epoch-1 = 0-based

        # append logs to history file
        logs = logs or {}
        for k, v in logs.items():
            history_dict.setdefault(k, []).append(float(v))
        with open(HISTORY_FILE, 'w') as f:
            json.dump(history_dict, f, indent=2)

full_cp = FullCheckpoint()
es = EarlyStopping(monitor='val_accuracy', patience=4,
                   restore_best_weights=True, mode='max', verbose=1)

# --------------------------------------------------------------
# 9. TRAIN (will continue from start_epoch)
# --------------------------------------------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    initial_epoch=start_epoch,
    callbacks=[full_cp, es],
    verbose=1
)

# --------------------------------------------------------------
# 10. FINAL SAVE (best model)
# --------------------------------------------------------------
final_path = f"{OUTPUT_DIR}/final_merged_model.keras"
model.save(final_path)
print(f"\nTRAINING FINISHED – final model saved to:\n{final_path}")

# --------------------------------------------------------------
# 11. QUICK TEST
# --------------------------------------------------------------
def predict_image(path):
    img = Image.open(path).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
    x = np.array(img, dtype=np.float32)[None, ...]
    rgb = rgb_preprocess(x)
    lap = laplacian_preprocess(x)
    p = model.predict([rgb, lap], verbose=0)[0,0]
    label = "Recaptured" if p > 0.5 else "Original"
    print(f"{os.path.basename(path)} → {label} ({p:.1%})")

# Test a recapture from your Android folder
predict_image("/content/drive/MyDrive/Nothing_2a/recaptures/IMG_20251002_164912794.jpg")

Mounted at /content/drive
Weights ready: /content/drive/MyDrive/efficientnetb0_notop.h5
Using existing merged dataset: /content/merged_dataset
Found 209 files belonging to 2 classes.
Using 168 files for training.
Found 209 files belonging to 2 classes.
Using 41 files for validation.
Train batches: 11 | Val batches: 3
No checkpoint → starting fresh
Loading EfficientNet backbone from Drive...
Backbone loaded
Training from epoch 1/15
Epoch 1/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 282s 11s/step - Precision: 0.4599 - Recall: 0.6279 - accuracy: 0.4849 - loss: 0.7316 - val_Precision: 0.5278 - val_Recall: 0.9048 - val_accuracy: 0.5366 - val_loss: 0.7058
Epoch 2/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 59s 5s/step - Precision: 0.6660 - Recall: 0.8260 - accuracy: 0.7129 - loss: 0.6074 - val_Precision: 0.5455 - val_Recall: 0.8571 - val_accuracy: 0.5610 - val_loss: 0.6921
Epoch 3/15
11/11 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - Precision: 0.8211 - Recall: 0.8578 - accuracy: 0.8544 - loss: 0.4745 - val_Precision: 0.6071 - 